<a href="https://colab.research.google.com/github/MahmoudSalama7/TwitterSentimentAnalysis/blob/main/Twitter_BoW.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import Libraries

In [73]:
import pandas as pd
import numpy as np

# Preprocessing

In [2]:
from google.colab import files

uploaded = files.upload()


Saving training.1600000.processed.noemoticon.csv to training.1600000.processed.noemoticon.csv


In [74]:
data=pd.read_csv("training.1600000.processed.noemoticon.csv",encoding='latin-1',header=None,names=['Sentiment','Id','Date','Query','User','Text'])

In [75]:
data

,Sentiment,Id,Date,Query,User,Text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
...,...,...,...,...,...,...
1599995,4,2193601966,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,AmandaMarie1028,Just woke up. Having no school is the best fee...
1599996,4,2193601969,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,TheWDBoards,TheWDB.com - Very cool to hear old Walt interv...
1599997,4,2193601991,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,bpbabe,Are you ready for your MoJo Makeover? Ask me f...
1599998,4,2193602064,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,tinydiamondz,Happy 38th Birthday to my boo of alll time!!! ...


In [76]:
data['Query'].nunique()

1

In [77]:
data=data.drop("Query",axis=1)

In [78]:
data.isnull().sum()

Sentiment    0
Id           0
Date         0
User         0
Text         0
dtype: int64

In [79]:
data=data.drop("Date",axis=1)

# Sampling Data

In [81]:
sample=data.sample(1000)

In [82]:
sample=sample.reset_index(drop=True)

In [83]:
sample

,Sentiment,Id,User,Text
0,0,2302082430,lilmonke,@deevyne241 why must you remind me of how bad ...
1,4,1969782749,hocwai,http://twitpic.com/68u2v nice weather ???...
2,0,2256304914,crixis,i need finger food I think i'm gonna have a...
3,4,1978384353,sQuint2718,I have a thousand bucks to spend carelessly. I...
4,0,2260221141,ddelilah,"@Amy617 i know mee2, this site is growing on m..."
...,...,...,...,...
995,4,2176220322,JeromeKoehler,"Finally home. Off tomorrow, but I have to fin..."
996,4,1881724202,benolicious,@Gtorinno yoi yoi uncle gave http://myloc.me/...
997,4,2189181993,Sjoem,@esmeeworld Making a Simlish version of Outta ...
998,4,2058708646,tundiscovered,2hrs and change...beautiful day for riding. Di...


# Remove Sensitive Words

In [84]:
from better_profanity import profanity

# Function to check if a text contains profanity
def contains_profanity(text):
  return profanity.contains_profanity(text)

# Apply profanity filter and create a new cleaned dataset
cleaned_data = []
for index, row in sample.iterrows():
  text = row['Text']
  if not contains_profanity(text):
    cleaned_data.append(row)

# Convert the cleaned data back to a DataFrame
cleaned_df = pd.DataFrame(cleaned_data, columns=sample.columns)



In [85]:
cleaned_df=cleaned_df.reset_index(drop=True)

In [86]:
cleaned_df

,Sentiment,Id,User,Text
0,0,2302082430,lilmonke,@deevyne241 why must you remind me of how bad ...
1,4,1969782749,hocwai,http://twitpic.com/68u2v nice weather ???...
2,0,2256304914,crixis,i need finger food I think i'm gonna have a...
3,4,1978384353,sQuint2718,I have a thousand bucks to spend carelessly. I...
4,0,2260221141,ddelilah,"@Amy617 i know mee2, this site is growing on m..."
...,...,...,...,...
917,0,2212429736,cndflybaby,@DianeDenmark didn't get to watch it on Saturday
918,4,2176220322,JeromeKoehler,"Finally home. Off tomorrow, but I have to fin..."
919,4,1881724202,benolicious,@Gtorinno yoi yoi uncle gave http://myloc.me/...
920,4,2189181993,Sjoem,@esmeeworld Making a Simlish version of Outta ...


# Remove Stopwords and Punctuations

In [88]:

import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
corpus = []
for i in range(0,922):
  review = re.sub('[^a-zA-Z]', ' ', cleaned_df['Text'][i])
  review = review.lower()
  review = review.split()
  ps = PorterStemmer()
  all_stopwords = stopwords.words('english')
  all_stopwords.remove('not')
  review = [ps.stem(word) for word in review if not word in set(all_stopwords)]
  review = ' '.join(review)
  corpus.append(review)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


# Build BoW model with SVM

In [89]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features = 1500)
X = cv.fit_transform(corpus).toarray()
y = cleaned_df['Sentiment']

In [90]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state = 0)

In [94]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

In [95]:
# Create an SVM classifier instance
svm = SVC(kernel='linear', C=1.0, random_state=42)

# Train the classifier on the scaled training data
svm.fit(X_train, y_train)


SVC(kernel='linear', random_state=42)

In [96]:
y_pred = svm.predict(X_test)


In [97]:
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.2f}")


Accuracy: 0.66
